## 웹크롤링

### 인터넷 접속 라이브러리 추가

- urllib.request 모듈로 작업
- html 소스로 별다른 문제 없습

In [6]:
from urllib.request import urlopen, Request

#도시별 날씨 검색함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page =  urlopen(url=url)

    text= page.read().decode('utf-8')
    text=text[text.find(f'>{city}</a'):]
    # 기온 가져오기
    for i in range(7):
        text= text[text.find('<td>')+1:]
        
    start=3
    end=text.find('</td>')
    current_temp= text[start:end]
    print(f'{city}의 현재기온은 {current_temp}˚C 입니다')

    #습도 가져오기
    for i in range(3):
        text= text[text.find('<td>')+1:]
        
    start=3
    end=text.find('</td>')
    current_temp= text[start:end]
    print(f'{city}의 현재 습도는 {current_temp}% 입니다.')


get_weather('부산')

부산의 현재기온은 12.2˚C 입니다
부산의 현재 습도는 24% 입니다.


### OpenAIP 크롤링

- 공공데이터 포털
    - http://www.data.go.kr
    - 부산광역시 버스정보안내기 현황
 
```python
response = request.get(total_url, verify= False)
```
- 문제점
    - https를 일반 request로 부르면 SSL 오류 발생
    - 해결하려면 외부모듈 requests를 사용, verify=False 옵션을 지정
    -  아래 셀의 19번 라인!

In [19]:
import requests
from urllib.request import urlopen, Request
from urllib.parse import quote,unquote,urlencode # 한글 URLencode 변환하는 함수
import json
import ssl

def getRequestUrl(url):
    req = Request(url)

    try:
        res=urlopen(req)
        if res.getcode()==200:
            return res.read().decode('utf-8')
    except Exception as e:
        print()
        return None

# stationName 정류소이름 
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = "?" + urlencode(

        {
            'serviceKey' : 'jmhj4AySTfuqupZqy/HnxlkTQs+iGPx391UTg2b1LoaAHtc6FzkpEnmerQYc5Oxvkb2klHsgnoTKB3nI0XllFg==',
            'pageNo' : '1',
            'numOfRows' : '10',
            'resultType' : type,
            'stationLoc' : stationName
        }
    )
    total_url= api_url+ queryString
    response = requests.get(total_url, verify= False)
    return response.text
 
try:
    result = getDataPortalSearch('동아대', 'json')
    json_data =  json.loads(result)
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

    for item in station_data:
        print(item)
except Exception as e:
    print('찾는 데이터가 없습니다')

c:\source\studypython2023\venv\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'stationNum': '10088', 'stationLoc': '동아대입구', 'lat': '35.11232061', 'lng': '128.9642117', 'addr': '사하구 하단2동 490-8', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '10090', 'stationLoc': '동아대입구', 'lat': '35.11260332', 'lng': '128.964507', 'addr': '사하구 하단2동 490-18', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '10092', 'stationLoc': '동아대입구', 'lat': '35.11362167', 'lng': '128.9636381', 'addr': '사하구 하단2동 산13-11', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '02056', 'stationLoc': '동아대부민캠퍼스', 'lat': '35.10605336', 'lng': '129.0199656', 'addr': '서구 부용동1가 62-7', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '02057', 'stationLoc': '동아대학교병원', 'lat': '35.11942772', 'lng': '129.0166502', 'addr': '서구 동대신동3가 13-5', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '02058', 'stationLoc': '동아대부민캠퍼스', 'lat': '35.105652', 'lng': '129.020154', 'addr': '서구 부민동 69-6', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '02180', 'stationLoc': '동아대부민캠퍼스', 'lat':

### OpenAPI로 가져온 데이터 지도 표시

```python
stop_str = '<h4>'+ item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>'+ item['addr']
```
- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}' 호환안됨!
    - 구식방법인 문자열 결합방식으로 해결 

In [21]:
import folium

if len(station_data) > 0:
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']
m = folium.Map(location=[center_lat, center_lng], zoom_start=13)

for item in station_data:
    stop_str = '<h4>'+ item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>'+ item['addr']
    iframe = folium.IFrame(stop_str)
    popup = folium.Popup(iframe, min_width=200, max_width=200) #팝업 사이즈 지정
    folium.Marker(location=[item['lat'], item['lng']], popup=popup,
                icon=folium.Icon(icon='pushpin')).add_to(m)
m

### BeautifulSoup (version4)
- 웹 크롤링을 편하게 해주는 도구(라이브러리)
```python
!pip install beautifulsoup4
```

In [22]:
!pip install beautifulsoup4

     -------------------------------------- 129.4/129.4 kB 7.4 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### 모듈 import

```python
from bs4 import Beautifulsoup
```

In [37]:
from bs4 import BeautifulSoup
import requests


url= 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'
res= requests.get(url)


if res.status_code == 200: # webpage OK
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    # print(soup)
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) , dl > dt > a')
    print(title.get_text())
else:
    print(f'Error : {res.status_code}')




제임스웹 우주망원경 질문이요

2022.07.13.
어제 밤에 제임스웹이 찍은 사진들이 공개됐는데요 그 사진들은 찍은 그대로가 아닌 보정작업을 거친 사진인가요? 당연히 그렇게 합니다 

#제임스웹우주망원경
#문제풀이


Q&A
								
									 > 천문우주학
|
답변수 1
							
							 추천수 0
							
								|
								
								
									답변 
								
								
									
									
									
									
									
									
										
									
								
								
									목우자(p850****)
								
								
							
							




In [46]:
from bs4 import BeautifulSoup
import requests


url= 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'
res= requests.get(url)


if res.status_code == 200: # webpage OK
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a')
    for title in titles:
        print(title.get_text())
else:
    print(f'Error : {res.status_code}')

제임스웹 우주망원경 질문이요
최고고도 부탁드립니다. 제임스웹 망원경이
제임스 웹 망원경 우주사진 원본 어디서... 
제임스 웹 망원경 촬영 원리
제임스 웹 망원경
제임스 웹 우주망원경이 찍은 첫 사진은... 
제임스 웹 망원경의 종류 알려주세요
제임스 웹 우주망원경
제임스웹 망원경
제임스 웹 우주 망원경 제작 기간이
